In [1]:
# In this script, we will read in CMIP5 abrupt4xCO2 and piControl air (3D) and surface (2D)
# temperature to calculate the temperature feedbacks, which can be separated into
# the Planck response/feedback and the lapse-rate feedback

# By: Ty Janoski
# Last edited: 05.28.20

In [2]:
# Import statements

import xarray as xr
import dask.array as da
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe
%matplotlib inline

In [3]:
# start by reading in the air temperature kernel
ta_kern = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/\
CAM5_kernels/t.kernel.plev.nc')
lats = ta_kern.lat
lons = ta_kern.lon
# some models have higher vertical resolution than the kernels. We only need 17 levels
plevs = [float(p) for p in ta_kern.plev]

# create a dataset with lat/lons for regridding
ds_out = xr.Dataset({'lat': (['lat'], lats),
                     'lon': (['lon'], lons),
                    }
                   )
# also need the surface temperature kernel
ts_kern = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/'+
                         'CAM5_kernels/ts.kernel.nc')

# tile kernels
ta_FLNT = da.tile(ta_kern.FLNT,(150,1,1,1))
ta_FLNTC = da.tile(ta_kern.FLNTC,(150,1,1,1))
ts_FLNT = da.tile(ts_kern.FLNT,(150,1,1))
ts_FLNTC = da.tile(ts_kern.FLNTC,(150,1,1))

In [4]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
for mod in models:
    print(mod)
    # read in air/surface temperature from last 50 years of piControl
    ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/ta_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).ta.isel(
        time=slice(-600,None)).sel(plev=plevs)
    ts = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/ts_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).ts.isel(time=slice(-600,None))
    
    # make climatology
    ta_clim = ta.groupby(ta.time.dt.month).mean(dim='time')
    ts_clim = ts.groupby(ts.time.dt.month).mean(dim='time')
    
    # read in first 150 years of abrupt4xCO2
    ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ta_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True,chunks={'time':300}).ta.isel(
        time=slice(None,1800)).sel(plev=plevs)
    ts = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ts_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True,chunks={'time':300}).ts.isel(
        time=slice(None,1800))
    
    
    # take difference abrupt4xCO2 - climatology
    dta = (ta - da.tile(ta_clim,(150,1,1,1))).load()
    dts = (ts - da.tile(ts_clim,(150,1,1))).load()
    
    del ta, ts, ta_clim, ts_clim
    
    # load in layer thickness, calculated in dz.ipynb
    dz = da.array(xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+
                           '_dz.nc',
                          use_cftime=True,chunks={'time':300}).dz.isel(
        time=slice(None,1800)))
    
    # regrid to CAM5 horizontal grid
    # issue: ACCESS models have different resolutions for 2D and 3D variables
    if(mod in ['ACCESS1-0','ACCESS1-3']):
        regridder_ta = xe.Regridder(dta,ds_out,'bilinear',periodic=True,reuse_weights=True)
        regridder_ts = xe.Regridder(dts,ds_out,'bilinear',periodic=True,reuse_weights=True)
        dta = regridder_ta(dta) * dz
        dts = regridder_ts(dts)
    else:
        regridder = xe.Regridder(dts,ds_out,'bilinear',periodic=True,reuse_weights=True)
        dta = regridder(dta) * dz
        dts = regridder(dts)
            
    # for planck, we need a vertically uniform temperature change corresponding to
    # the change in surface temperature
    dt_uniform = xr.broadcast(dts,dta)[0]
    dt_uniform = dt_uniform.transpose('time','plev','lat','lon') * dz
    
    del dz
    
    # now we can calculate feedbacks
    # planck = uniform change in temp * longwave air temp kernel + change in surface temp * longwave surface temp kernel
    planck = ((dt_uniform * ta_FLNT)/-100).sum(dim='plev') + (
        dts * ts_FLNT * -1)
    print(planck.time)
    planck.rename('planck_LWAS').to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+
                                           mod+'_planck.nc')
    del planck
    # lapse rate = difference between change in air temperature and uniform change in temperature *
    # longwave air temperature kernel
    lapse = ((dta - dt_uniform) * ta_FLNT/-100).sum(dim='plev')
    lapse.rename('lapse_LWAS').to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+
                                           mod+'_lapse.nc')
    del lapse
    del dt_uniform
    
    # also calculate total temperature feedbacks for calculating cloud feedbacks later
    all_sky = ((dta * ta_FLNT)/-100).sum(dim='plev') + (
        dts * ts_FLNT * -1)
    clear_sky = ((dta * ta_FLNTC)/-100).sum(dim='plev') + (
        dts * ts_FLNTC * -1)
    out = xr.merge([all_sky.rename('temp_LWAS'),clear_sky.rename('temp_LWCS')])
    out.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+
                                           mod+'_temp.nc')
    del all_sky,clear_sky,out
    
    dts.rename('dTs').to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_dTs.nc')

MRI-CGCM3
Reuse existing file: bilinear_160x320_192x288_peri.nc
NorESM1-M
Reuse existing file: bilinear_96x144_192x288_peri.nc
